In [1]:
import pandas as pd

In [2]:
m = pd.read_csv('./UnitedStates_M.csv')
q = pd.read_csv('./UnitedStates_Q.csv')

In [3]:
m.head(5)

index   Core CPI  Unemployment rate  Stock Markets
0  2008M01 [2008M01]  96.843052                5.0     119.738713
1  2008M02 [2008M02]  97.173832                4.9     118.606518
2  2008M03 [2008M03]  97.628200                5.1     116.451180
3  2008M04 [2008M04]  97.715893                5.0     120.870484
4  2008M05 [2008M05]  97.770871                5.4     122.358835

In [4]:
m_index = m['index'].apply(lambda x: x[:7])
m_date = pd.to_datetime(m_index, format = '%YM%m')
m.loc[:,'index'] = m_date
m.head()

index   Core CPI  Unemployment rate  Stock Markets
0 2008-01-01  96.843052                5.0     119.738713
1 2008-02-01  97.173832                4.9     118.606518
2 2008-03-01  97.628200                5.1     116.451180
3 2008-04-01  97.715893                5.0     120.870484
4 2008-05-01  97.770871                5.4     122.358835

In [5]:
q.head()

index  GDP in  US$
0  2008Q1 [2008Q1]      3662800
1  2008Q2 [2008Q2]      3701400
2  2008Q3 [2008Q3]      3708800
3  2008Q4 [2008Q4]      3639900
4  2009Q1 [2009Q1]      3598600

In [46]:
q_index = q['index'].apply(lambda x: x.replace('Q', 'M')[:6])
q_index = q_index.apply(lambda x: x[:-1] + str(int(x[-1])*3-2))
q_date = pd.to_datetime(q_index, format = '%YM%m')
q.loc[:,'index'] = q_date
q.head()

index  GDP in  US$
0 2008-01-01      3662800
1 2008-04-01      3701400
2 2008-07-01      3708800
3 2008-10-01      3639900
4 2009-01-01      3598600

In [56]:
print(m.shape)
print(q.shape)

(151, 4)
(51, 2)


In [61]:
overall = m.merge(q, on = 'index', how = 'left')

In [64]:
overall.fillna(method='ffill', inplace = True)

In [65]:
overall

index    Core CPI  Unemployment rate  Stock Markets  GDP in  US$
0   2008-01-01   96.843052                5.0     119.738713    3662800.0
1   2008-02-01   97.173832                4.9     118.606518    3662800.0
2   2008-03-01   97.628200                5.1     116.451180    3662800.0
3   2008-04-01   97.715893                5.0     120.870484    3701400.0
4   2008-05-01   97.770871                5.4     122.358835    3701400.0
..         ...         ...                ...            ...          ...
146 2020-03-01  121.457975                4.4     216.186728    5384900.0
147 2020-04-01  120.902284               14.7     222.456085    5384900.0
148 2020-05-01  120.770517               13.3     231.787567    5384900.0
149 2020-06-01  120.999064               11.1     248.893940    5384900.0
150 2020-07-01  120.999064               11.1     251.806176    5384900.0

[151 rows x 5 columns]

In [75]:
def combine(m_data, q_data, country_name):
    m_index = m_data['index'].apply(lambda x: x[:7])
    m_date = pd.to_datetime(m_index, format = '%YM%m')
    m_data.loc[:,'index'] = m_date
    
    q_index = q_data['index'].apply(lambda x: x.replace('Q', 'M')[:6])
    q_index = q_index.apply(lambda x: x[:-1] + str(int(x[-1])*3-2))
    q_date = pd.to_datetime(q_index, format = '%YM%m')
    q_data.loc[:,'index'] = q_date
    
    combined = m_data.merge(q_data, on = 'index', how = 'left')
    combined.fillna(method='ffill', inplace = True)
    
    combined['country'] = country_name
    
    return combined

In [76]:
country_names = ['Brazil', 'Canada', 'HongKong', 'Japan', 'Mexico',
                'Russia', 'SouthAfrica', 'Thailand', 'UnitedKingdom',
                'UnitedStates']

In [86]:
GDP_data = pd.DataFrame(columns= ['index',	'Core CPI', 'Unemployment rate', 
                                  'Stock Markets', 'GDP in US$', 'country'])

In [87]:
for country in country_names:
    m_path = './EDI_dataset/' + country + '_M.csv'
    q_path = './EDI_dataset/' + country + '_Q.csv'
    
    m_data = pd.read_csv(m_path)
    q_data = pd.read_csv(q_path)
    
    combined_data = combine(m_data, q_data, country)
    
    GDP_data =  GDP_data.append(combined_data)

In [99]:
GDP_data.drop(['GDP in US$', 'level_0_x', 'level_0_y'], axis = 1, inplace = True)

In [101]:
GDP_data.to_csv('combined_indicators.csv')

In [116]:
GDP_data['index'].unique()[0]

numpy.datetime64('2008-01-01T00:00:00.000000000')

In [118]:
GDP_data['index'].unique()[-1]

numpy.datetime64('2020-07-01T00:00:00.000000000')

In [104]:
final_data = pd.read_csv('./final_data.csv')

In [114]:
final_data['date'].unique()[0]

'2020-02-15'

In [112]:
final_data['date'].unique()[-1]

'2020-10-27'

In [119]:
GDP_data

index    Core CPI  Unemployment rate  Stock Markets       country  \
0   2008-01-01   90.153723          11.157871      87.101121        Brazil   
1   2008-02-01   90.694644          11.312779      94.025339        Brazil   
2   2008-03-01   91.111840          10.791905      93.490682        Brazil   
3   2008-04-01   91.631178          10.700540      98.750100        Brazil   
4   2008-05-01   92.217617           9.939990     111.320716        Brazil   
..         ...         ...                ...            ...           ...   
146 2020-03-01  121.457975           4.400000     216.186728  UnitedStates   
147 2020-04-01  120.902284          14.700000     222.456085  UnitedStates   
148 2020-05-01  120.770517          13.300000     231.787567  UnitedStates   
149 2020-06-01  120.999064          11.100000     248.893940  UnitedStates   
150 2020-07-01  120.999064          11.100000     251.806176  UnitedStates   

     GDP in  US$  
0      3662800.0  
1      3662800.0  
2      3662800.0  
3      3701400.0  
4      3701400.0  
..           ...  
146    5384900.0  
147    5384900.0  
148    5384900.0  
149    5384900.0  
150    5384900.0  

[1510 rows x 6 columns]

In [166]:
interpolated_df = pd.DataFrame(columns=['country', 'EDI'])

In [169]:
for country in country_names:
    temp_df = edi.loc[edi['country'] == country, ]
    temp_df.set_index('index', inplace = True)
    temp_df.index = pd.to_datetime(temp_df.index)
    temp_df = temp_df.reindex(time_index)
    temp_df = temp_df.interpolate().fillna(method = 'ffill')
    temp_df = temp_df.reset_index()
    
    interpolated_df = interpolated_df.append(temp_df)

In [176]:
interpolated_df.replace({'UnitedKingdom':'United Kingdom',
                        'UnitedStates':'Untied States',
                        'SouthAfrica':'South Africa'}, inplace = True)

In [179]:
interpolated_df = interpolated_df.loc[interpolated_df['country'] != 'HongKong', ]

In [181]:
interpolated_df.to_csv('interpolated_edi.csv')

In [163]:
edi['country'].unique()

array(['Brazil', 'Canada', 'HongKong', 'Japan', 'Mexico', 'Russia',
       'SouthAfrica', 'Thailand', 'UnitedKingdom', 'UnitedStates'],
      dtype=object)

In [126]:
time_index = pd.date_range(start = '2/1/2020', end = '7/1/2020')

In [127]:
time_index

DatetimeIndex(['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10',
               ...
               '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25',
               '2020-06-26', '2020-06-27', '2020-06-28', '2020-06-29',
               '2020-06-30', '2020-07-01'],
              dtype='datetime64[ns]', length=152, freq='D')

In [120]:
edi = pd.read_csv('./EDI_indicators.csv')

In [149]:
temp_df = edi.loc[edi['country'] == 'Brazil', ]
temp_df.set_index('index', inplace = True)
temp_df.index = pd.to_datetime(temp_df.index)

In [150]:
temp_df.index

DatetimeIndex(['2008-01-01', '2008-02-01', '2008-03-01', '2008-04-01',
               '2008-05-01', '2008-06-01', '2008-07-01', '2008-08-01',
               '2008-09-01', '2008-10-01',
               ...
               '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01',
               '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01',
               '2020-06-01', '2020-07-01'],
              dtype='datetime64[ns]', name='index', length=151, freq=None)

In [133]:
time_index

DatetimeIndex(['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10',
               ...
               '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25',
               '2020-06-26', '2020-06-27', '2020-06-28', '2020-06-29',
               '2020-06-30', '2020-07-01'],
              dtype='datetime64[ns]', length=152, freq='D')

In [156]:
temp_df = temp_df.reindex(time_index)

In [161]:
temp_df.interpolate().fillna(method = 'ffill')

country       EDI
2020-02-01  Brazil  4.678294
2020-02-02  Brazil  4.653117
2020-02-03  Brazil  4.627939
2020-02-04  Brazil  4.602762
2020-02-05  Brazil  4.577584
...            ...       ...
2020-06-27  Brazil  3.939519
2020-06-28  Brazil  3.941429
2020-06-29  Brazil  3.943339
2020-06-30  Brazil  3.945248
2020-07-01  Brazil  3.947158

[152 rows x 2 columns]

In [165]:
temp_df.reset_index()

index country       EDI
0   2020-02-01  Brazil  4.678294
1   2020-02-02     NaN       NaN
2   2020-02-03     NaN       NaN
3   2020-02-04     NaN       NaN
4   2020-02-05     NaN       NaN
..         ...     ...       ...
147 2020-06-27     NaN       NaN
148 2020-06-28     NaN       NaN
149 2020-06-29     NaN       NaN
150 2020-06-30     NaN       NaN
151 2020-07-01  Brazil  3.947158

[152 rows x 3 columns]